In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras


2024-02-03 04:36:37.086692: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-03 04:36:37.086787: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-03 04:36:37.211184: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



Inside each attention head is a **Scaled Dot Product Self-Attention** operation as we covered in the slides. Given *queries*, *keys*, and *values*, the operation returns a new "mix" of the values.

$$Attention(Q, K, V) = softmax(\frac{QK^T)}{\sqrt{d_k}})V$$

The following function implements this and also takes a mask to account for padding and for masking future tokens for decoding (i.e. **look-ahead mask**). We'll cover masking later in the notebook.

In [4]:
import numpy as np

# Example sentence
sentence = "ChatGPT is a powerful language model developed by OpenAI."

# Tokenize the sentence (for simplicity, splitting by whitespace)
tokens = sentence.split()

# Embedding dimension
embed_dim = 4

# Generate random word embeddings for query, keys, and values
query = np.random.rand(len(tokens), embed_dim)
keys = np.random.rand(len(tokens), embed_dim)
values = np.random.rand(len(tokens), embed_dim)

print("Tokens:", tokens)
print("\nQuery (representing what we want to focus on):\n", query)
print("\nKeys (used to compute attention scores):\n", keys)
print("\nValues (actual content to be weighted by attention):\n", values)


Tokens: ['ChatGPT', 'is', 'a', 'powerful', 'language', 'model', 'developed', 'by', 'OpenAI.']

Query (representing what we want to focus on):
 [[0.46847462 0.55728415 0.05759386 0.50289439]
 [0.88920322 0.53325944 0.93581348 0.22094825]
 [0.31026488 0.10372769 0.39506193 0.73457544]
 [0.69338079 0.22518227 0.44290632 0.94135132]
 [0.46429722 0.59473349 0.72871288 0.5137612 ]
 [0.24564315 0.7080508  0.89020335 0.11812936]
 [0.89369661 0.45697618 0.31179819 0.57846279]
 [0.48877444 0.86706101 0.42218196 0.71727902]
 [0.5987527  0.7469727  0.72846239 0.41204846]]

Keys (used to compute attention scores):
 [[0.48701525 0.14010646 0.20521205 0.81737574]
 [0.33358357 0.6259162  0.51800412 0.69906038]
 [0.59493203 0.05543959 0.06590504 0.97363518]
 [0.38382367 0.54254271 0.73150523 0.8711893 ]
 [0.36934798 0.30439262 0.8468169  0.92941526]
 [0.55704418 0.99395462 0.30430115 0.32216649]
 [0.41118298 0.52651466 0.01441199 0.09073061]
 [0.90034636 0.10263335 0.90043922 0.55586274]
 [0.46346165 0

In [5]:
def scaled_dot_product_attention(query,key,values,mask=None):
    
    key_dim=tf.cast(tf.shape(key)[-1], tf.float32)
    scaled_scores= tf.matmul(query,key,transpose_b=True) / np.sqrt (key_dim)
    if mask is not None:
        scaled_scores = tf.where(mask==0, -np.inf, scaled_scores)
    
    softmax=tf.keras.layers.Softmax()
    weights=softmax(scaled_scores)
    return tf.matmul(weights,values),weights


In [6]:
seq_len=3
embed_dim=4
queries=np.random.rand(seq_len,embed_dim)
keys=np.random.rand(seq_len,embed_dim)
values=np.random.rand(seq_len,embed_dim)



In [8]:
output, attn_weights = scaled_dot_product_attention(queries, keys, values)

print("Output\n", output, "\n")
print("Weights\n", attn_weights)

Output
 tf.Tensor(
[[0.4759968  0.35926855 0.5374737  0.59249526]
 [0.46812192 0.36584187 0.53658634 0.5869098 ]
 [0.46033582 0.36937323 0.5173398  0.5811532 ]], shape=(3, 4), dtype=float32) 

Weights
 tf.Tensor(
[[0.34181294 0.37451047 0.28367662]
 [0.33770812 0.38685325 0.27543864]
 [0.3794669  0.37407982 0.24645327]], shape=(3, 3), dtype=float32)


## multi head Self attention

In [9]:
batch_size = 1
seq_len = 3
embed_dim = 12
num_heads = 3 # eti times chai Q K V calculate hunxa
head_dim = embed_dim // num_heads

print(f"Dimension of each head: {head_dim}")

Dimension of each head: 4


In [10]:
x = np.random.rand(batch_size, seq_len, embed_dim).round(1) # round(1) bhaneko round up gareko
print("Input shape: ", x.shape, "\n")
print("Input:\n", x)


Input shape:  (1, 3, 12) 

Input:
 [[[0.6 0.8 0.2 0.6 0.6 0.8 0.6 0.4 0.4 0.5 0.8 1. ]
  [0.8 0.4 0.9 0.8 0.5 0.6 0.5 0.3 0.3 0.5 0.4 0.4]
  [0.8 0.  0.9 0.7 0.  0.2 0.3 0.3 0.7 0.4 0.4 1. ]]]


In [11]:
# query weights for each head
wq0=np.random.rand(embed_dim,head_dim).round(1)
wq1=np.random.rand(embed_dim,head_dim).round(1)
wq2=np.random.rand(embed_dim,head_dim).round(1)

# key weights for each head
wk0=np.random.rand(embed_dim,head_dim).round(1)
wk1=np.random.rand(embed_dim,head_dim).round(1)
wk2=np.random.rand(embed_dim,head_dim).round(1)

# value weights for each head
wv0=np.random.rand(embed_dim,head_dim).round(1)
wv1=np.random.rand(embed_dim,head_dim).round(1)
wv2=np.random.rand(embed_dim,head_dim).round(1)

In [12]:
print("wq0:\n",wq0)
print("wq1:\n",wq1)
print("wq2:\n",wq2)

wq0:
 [[0.8 0.5 0.5 0.9]
 [0.2 0.2 0.7 0.7]
 [0.2 0.3 0.3 0.6]
 [0.6 0.  0.5 0.8]
 [0.8 0.6 0.6 0.4]
 [0.5 0.3 0.7 0.4]
 [0.6 0.1 0.8 1. ]
 [0.2 0.9 0.7 0.4]
 [0.3 0.1 0.2 0. ]
 [0.9 0.6 0.7 0.2]
 [0.  0.3 0.8 0.8]
 [0.5 0.5 0.3 0.9]]
wq1:
 [[0.3 0.8 0.1 0.7]
 [0.4 0.5 0.7 0.3]
 [0.7 0.7 0.1 0.1]
 [0.1 0.3 1.  0.5]
 [0.5 0.4 0.6 0.8]
 [1.  0.1 0.5 0.3]
 [0.2 0.3 0.5 0.2]
 [0.  1.  0.  0.4]
 [0.3 0.4 0.1 0.5]
 [0.2 0.8 0.6 0.3]
 [0.3 0.1 0.7 0.9]
 [0.9 0.3 0.4 0.4]]
wq2:
 [[0.7 0.3 1.  0.2]
 [0.4 0.1 0.7 0.9]
 [0.8 0.3 0.6 0.2]
 [0.  0.7 0.1 0.2]
 [0.5 0.8 0.  0.1]
 [0.5 0.5 1.  0.3]
 [0.6 0.5 0.3 0. ]
 [0.8 0.7 0.1 0.9]
 [0.3 0.7 0.3 0.5]
 [0.6 0.3 0.7 0.7]
 [0.3 1.  0.6 0.2]
 [0.4 0.6 0.2 0.2]]


# Generating queries keys and values for 3 heads

In [13]:
q0=np.dot(x,wq0)
k0=np.dot(x,wk0)
v0=np.dot(x,wv0)

q1=np.dot(x,wq1)
k1=np.dot(x,wk1)
v1=np.dot(x,wv1)

q2=np.dot(x,wq2)
k2=np.dot(x,wk2)
v2=np.dot(x,wv2)

In [11]:
q0.shape

(1, 3, 4)

## Now we pass them to self attention operation

In [14]:
out0,attn_weights0=scaled_dot_product_attention(q0,k0,v0)

In [15]:
out1,_=scaled_dot_product_attention(q1,k1,v1)
out2,_=scaled_dot_product_attention(q1,k1,v1)

In [16]:
combined_out_a=np.concatenate((out0,out1,out2),axis=-1)

In [17]:
combined_out_a.shape

(1, 3, 12)

In [18]:
combined_out_a

array([[[3.342269 , 4.4451604, 3.4119306, 3.6446478, 2.7585423,
         3.8347876, 3.3595817, 3.9946008, 2.7585423, 3.8347876,
         3.3595817, 3.9946008],
        [3.330148 , 4.4309726, 3.4041188, 3.6375225, 2.728777 ,
         3.8075252, 3.357984 , 3.9665766, 2.728777 , 3.8075252,
         3.357984 , 3.9665766],
        [3.3049078, 4.4025617, 3.3879461, 3.6222346, 2.701246 ,
         3.780736 , 3.3554769, 3.938585 , 2.701246 , 3.780736 ,
         3.3554769, 3.938585 ]]], dtype=float32)

# Lets perform this by using a single query matrix,singlekeyweightmatrix and single value weight matrix

In [17]:
wq0

array([[0. , 0.9, 0.8, 0.1],
       [1. , 0.6, 0.1, 0.8],
       [0.1, 0.4, 0.2, 0.4],
       [0.1, 0.3, 0.9, 0.5],
       [0.9, 0.7, 0.7, 0.3],
       [0.3, 0.3, 0.2, 0.2],
       [0.4, 0. , 0.5, 0.8],
       [1. , 0. , 0.3, 0.7],
       [0.9, 0.8, 0.5, 0.4],
       [0.6, 0.8, 1. , 1. ],
       [0.9, 0.6, 0.4, 0.9],
       [0.3, 0.6, 0.3, 0.1]])

In [19]:
wq=np.concatenate((wq0,wq1,wq2),axis=1)
wk=np.concatenate((wk0,wk1,wk2),axis=1)
wv=np.concatenate((wv0,wv1,wv2),axis=1)

wv.shape

(12, 12)

In [20]:
q_s=np.dot(x,wq)
k_s=np.dot(x,wk)
v_s=np.dot(x,wv)

In [20]:
k_s.shape # 1 bhaneko batch size 3 bhaneko sequence length 12 bhaneko embed_dim


(1, 3, 12)

In [21]:
qs_reshaped=tf.reshape(q_s,(batch_size,seq_len,num_heads,head_dim))
ks_reshaped=tf.reshape(k_s,(batch_size,seq_len,num_heads,head_dim))
vs_reshaped=tf.reshape(v_s,(batch_size,seq_len,num_heads,head_dim))

In [22]:
print(qs_reshaped)
print(ks_reshaped)
print(vs_reshaped)

tf.Tensor(
[[[[3.43 2.62 4.27 4.66]
   [3.28 3.04 3.6  3.45]
   [3.34 4.03 3.51 2.57]]

  [[3.18 2.2  3.53 4.02]
   [2.73 3.1  2.87 2.79]
   [3.2  3.27 3.24 2.02]]

  [[2.65 1.96 2.65 3.62]
   [2.51 2.83 2.11 2.47]
   [2.77 3.07 2.66 1.72]]]], shape=(1, 3, 3, 4), dtype=float64)
tf.Tensor(
[[[[3.98 2.06 3.29 2.76]
   [3.89 2.78 4.17 3.38]
   [2.94 3.6  3.54 4.09]]

  [[3.42 2.2  2.7  2.56]
   [3.15 2.68 3.85 2.71]
   [2.99 3.57 3.34 3.74]]

  [[3.01 1.89 1.9  2.43]
   [2.44 2.88 3.12 2.65]
   [2.2  2.84 2.72 3.83]]]], shape=(1, 3, 3, 4), dtype=float64)
tf.Tensor(
[[[[3.37 4.48 3.43 3.66]
   [2.8  3.87 3.36 4.03]
   [2.71 3.76 4.2  3.31]]

  [[3.05 4.06 3.22 3.49]
   [1.9  3.15 3.38 3.32]
   [2.09 3.89 3.94 2.76]]

  [[2.29 3.53 2.76 2.9 ]
   [2.18 2.91 3.07 2.93]
   [2.07 3.64 2.87 2.52]]]], shape=(1, 3, 3, 4), dtype=float64)


In [23]:
qs_transposed=tf.transpose(qs_reshaped,perm=[0,2,1,3]).numpy()
ks_transposed = tf.transpose(ks_reshaped, perm=[0, 2, 1, 3]).numpy()
vs_transposed = tf.transpose(vs_reshaped, perm=[0, 2, 1, 3]).numpy()
print(f"Keys for all heads in a single matrix {ks_transposed.shape}: \n", ks_transposed, "\n")
print(f"Values for all heads in a single matrix {v_s.shape}: \n", vs_transposed)

Keys for all heads in a single matrix (1, 3, 3, 4): 
 [[[[3.98 2.06 3.29 2.76]
   [3.42 2.2  2.7  2.56]
   [3.01 1.89 1.9  2.43]]

  [[3.89 2.78 4.17 3.38]
   [3.15 2.68 3.85 2.71]
   [2.44 2.88 3.12 2.65]]

  [[2.94 3.6  3.54 4.09]
   [2.99 3.57 3.34 3.74]
   [2.2  2.84 2.72 3.83]]]] 

Values for all heads in a single matrix (1, 3, 12): 
 [[[[3.37 4.48 3.43 3.66]
   [3.05 4.06 3.22 3.49]
   [2.29 3.53 2.76 2.9 ]]

  [[2.8  3.87 3.36 4.03]
   [1.9  3.15 3.38 3.32]
   [2.18 2.91 3.07 2.93]]

  [[2.71 3.76 4.2  3.31]
   [2.09 3.89 3.94 2.76]
   [2.07 3.64 2.87 2.52]]]]


In [24]:
all_heads_output, all_attn_weights = scaled_dot_product_attention(qs_transposed, 
                                                                  ks_transposed, 
                                                                  vs_transposed)
print("Self attention output:\n", all_heads_output)

Self attention output:
 tf.Tensor(
[[[[3.342269  4.4451604 3.4119306 3.6446478]
   [3.330148  4.4309726 3.4041188 3.6375225]
   [3.3049078 4.4025617 3.3879461 3.6222346]]

  [[2.7585423 3.8347876 3.3595817 3.9946008]
   [2.728777  3.8075252 3.357984  3.9665766]
   [2.701246  3.780736  3.3554769 3.938585 ]]

  [[2.5115862 3.7997582 4.1090894 3.1323748]
   [2.4918818 3.8027678 4.0961404 3.113914 ]
   [2.4745855 3.8046224 4.081486  3.097021 ]]]], shape=(1, 3, 3, 4), dtype=float32)


In [25]:
combined_out_b = tf.reshape(tf.transpose(all_heads_output, perm=[0, 2, 1, 3]), 
                            shape=(batch_size, seq_len, embed_dim))
print("Final output from using single query, key, value matrices:\n", 
      combined_out_b, "\n")
print("Final output from using separate query, key, value matrices per head:\n", 
      combined_out_a)

Final output from using single query, key, value matrices:
 tf.Tensor(
[[[3.342269  4.4451604 3.4119306 3.6446478 2.7585423 3.8347876 3.3595817
   3.9946008 2.5115862 3.7997582 4.1090894 3.1323748]
  [3.330148  4.4309726 3.4041188 3.6375225 2.728777  3.8075252 3.357984
   3.9665766 2.4918818 3.8027678 4.0961404 3.113914 ]
  [3.3049078 4.4025617 3.3879461 3.6222346 2.701246  3.780736  3.3554769
   3.938585  2.4745855 3.8046224 4.081486  3.097021 ]]], shape=(1, 3, 12), dtype=float32) 

Final output from using separate query, key, value matrices per head:
 [[[3.342269  4.4451604 3.4119306 3.6446478 2.7585423 3.8347876 3.3595817
   3.9946008 2.7585423 3.8347876 3.3595817 3.9946008]
  [3.330148  4.4309726 3.4041188 3.6375225 2.728777  3.8075252 3.357984
   3.9665766 2.728777  3.8075252 3.357984  3.9665766]
  [3.3049078 4.4025617 3.3879461 3.6222346 2.701246  3.780736  3.3554769
   3.938585  2.701246  3.780736  3.3554769 3.938585 ]]]


In [26]:
class MultiHeadSelfAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadSelfAttention, self).__init__()
    self.d_model = d_model
    self.num_heads = num_heads

    self.d_head = self.d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(self.d_model)
    self.wk = tf.keras.layers.Dense(self.d_model)
    self.wv = tf.keras.layers.Dense(self.d_model)

    # Linear layer to generate the final output.
    self.dense = tf.keras.layers.Dense(self.d_model)
  
  def split_heads(self, x):
    batch_size = x.shape[0]

    split_inputs = tf.reshape(x, (batch_size, -1, self.num_heads, self.d_head))
    return tf.transpose(split_inputs, perm=[0, 2, 1, 3])
  
  def merge_heads(self, x):
    batch_size = x.shape[0]

    merged_inputs = tf.transpose(x, perm=[0, 2, 1, 3])
    return tf.reshape(merged_inputs, (batch_size, -1, self.d_model))

  def call(self, q, k, v, mask):
    qs = self.wq(q)
    ks = self.wk(k)
    vs = self.wv(v)
    qs = self.split_heads(qs)
    ks = self.split_heads(ks)
    vs = self.split_heads(vs)

    output, attn_weights = scaled_dot_product_attention(qs, ks, vs, mask)
    output = self.merge_heads(output)

    return self.dense(output), attn_weights


In [27]:
mhsa = MultiHeadSelfAttention(12, 3)

output, attn_weights = mhsa(x, x, x, None)
print(f"MHSA output{output.shape}:")
print(output)

MHSA output(1, 3, 12):
tf.Tensor(
[[[-0.14611931 -0.00353573 -0.33451563 -0.12995589 -0.14621437
   -0.17226407  0.46141407  0.33941475  0.25833526 -0.8124912
    0.12275562 -0.5625761 ]
  [-0.15156002 -0.00864987 -0.3424868  -0.11953118 -0.1656064
   -0.16633493  0.47230232  0.32969716  0.26439598 -0.805861
    0.13228002 -0.56805676]
  [-0.14889887 -0.00572135 -0.35030594 -0.14110234 -0.16682968
   -0.1949113   0.4835528   0.35853136  0.26331925 -0.79940844
    0.10877711 -0.58139986]]], shape=(1, 3, 12), dtype=float32)


In [28]:
def multi_head_attention(num_heads,embed_dim,mask=None):
    d_head=embed_dim//num_heads
    wq=tf.keras.layers.Dense(embed_dim)
    wk=tf.keras.layers.Dense(embed_dim)
    wv=tf.keras.layers.Dense(embed_dim)
    dense=tf.keras.layers.Dense(embed_dim)
    
    qs=wq(embed_dim)
    ks=wk(embed_dim)
    vs=wv(embed_dim)
    
    def split_heads(x,d_head):
        batch_size=x.shape[0]
        split_inputs=tf.reshape(x,(batch_size,-1,num_heads,d_head))
        return tf.transpose(split_inputs,perms=[0,2,1,3])
    def merge_heads(x):

        batch_size = x.shape[0]
        merged_inputs = tf.transpose(x, perm=[0, 2, 1, 3])
        
        return tf.reshape(merged_inputs, (batch_size, -1, d_model))
    qs=split_heads(qs,d_head)
    ks=split_heads(ks,d_head)    
    vs=split_heads(vs,d_head)    
    
    output, attn_weights = scaled_dot_product_attention(qs, ks, vs, mask)
    output = merge_heads(output)
    return dense(output),attn_weights

In [29]:
def multiheads(q,k,v,embed_dims,num_heads,mask=None):
    d_heads=embed_dims//num_heads
    wq=tf.keras.layers.Dense(embed_dims)
    wk=tf.keras.layers.Dense(embed_dims)
    wv=tf.keras.layers.Dense(embed_dims)
    dense=tf.keras.layers.Dense(embed_dims)
    qs=wq(embed_dims)
    ks=wk(embed_dims)
    vs=wv(embed_dims)
    def split_heads(x,d_heads):
        batch_size=x.shape[0]
        split_inputs=tf.reshape(x,(batch_size,-1,num_heads,d_head))
        return tf.transpose(split_inputs,perms=[0,2,1,3])
    def merge_heads(x,d_heads):
        batch_size=x.shape[0]
        merged_inputs = tf.reshape(x, perm=[0, 2, 1, 3])
        return tf.reshape(merged_inputs, (batch_size, -1, d_model))
    qs=split_heads(qs,d_heads)
    ks=split_heads(ks,d_heads)    
    vs=split_heads(vs,d_heads)    
    output,attn_weights=scaled_dot_product_attention(qs,ks,vs)
    return dense(output),attn_weights

In [30]:
def multiheads(embed_dims, num_heads, mask=None):
    d_heads = embed_dims // num_heads

    wq = tf.keras.layers.Dense(embed_dims)
    wk = tf.keras.layers.Dense(embed_dims)
    wv = tf.keras.layers.Dense(embed_dims)
    dense = tf.keras.layers.Dense(embed_dims)

    qs = wq(embed_dims)
    ks = wk(embed_dims)
    vs = wv(embed_dims)

    def split_heads(x, d_heads):
        batch_size = tf.shape(x)[0]
        split_inputs = tf.reshape(x, (batch_size, -1, num_heads, d_heads))
        return tf.transpose(split_inputs, perm=[0, 2, 1, 3])

    def merge_heads(x, d_heads):
        batch_size = tf.shape(x)[0]
        merged_inputs = tf.reshape(x, (batch_size, -1, embed_dims))
        return merged_inputs

    qs = split_heads(qs, d_heads)
    ks = split_heads(ks, d_heads)
    vs = split_heads(vs, d_heads)

    output, attn_weights = scaled_dot_product_attention(qs, ks, vs, mask)

    output = merge_heads(output, d_heads)
    output = dense(output)

    return output, attn_weights

In [34]:
mhsa = MultiHeadSelfAttention(12, 3)

output, attn_weights = mhsa(x, x, x, None)
print(f"MHSA output{output.shape}:")
print(output)

MHSA output(1, 3, 12):
tf.Tensor(
[[[-0.5498718   0.06832296 -0.00361164 -0.5559154  -0.18834695
   -0.18672106 -0.2933884  -0.35499278  0.8286581   0.5925399
   -0.5859465  -0.21757594]
  [-0.54551506  0.05759305 -0.00373295 -0.5550678  -0.18249266
   -0.17363487 -0.28998953 -0.35304824  0.82220197  0.5951256
   -0.5906154  -0.22028092]
  [-0.54178023  0.06347764 -0.00188243 -0.5565109  -0.17512456
   -0.17662172 -0.29091316 -0.35038674  0.8201754   0.592192
   -0.58951706 -0.21935263]]], shape=(1, 3, 12), dtype=float32)


In [35]:
def feed_forward_network(d_model, hidden_dim):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(hidden_dim, activation='relu'),
      tf.keras.layers.Dense(d_model)
  ])

In [37]:
class EncoderBlock(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, hidden_dim, dropout_rate=0.1):
    super(EncoderBlock, self).__init__()

    self.mhsa = MultiHeadSelfAttention(d_model, num_heads)
    self.ffn = feed_forward_network(d_model, hidden_dim)

    self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
    self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    self.layernorm1 = tf.keras.layers.LayerNormalization()
    self.layernorm2 = tf.keras.layers.LayerNormalization()
  
  def call(self, x, training, mask):
    mhsa_output, attn_weights = self.mhsa(x, x, x, mask)
    mhsa_output = self.dropout1(mhsa_output, training=training)
    mhsa_output = self.layernorm1(x + mhsa_output)

    ffn_output = self.ffn(mhsa_output)
    ffn_output = self.dropout2(ffn_output, training=training)
    output = self.layernorm2(mhsa_output + ffn_output)

    return output, attn_weights


In [41]:
!pip install BPEmb
from bpemb import BPEmb
# Load the English tokenizer.
bpemb_en = BPEmb(lang="en")

downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.model


100%|██████████| 400869/400869 [00:00<00:00, 706676.81B/s]


downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.d100.w2v.bin.tar.gz


100%|██████████| 3784656/3784656 [00:01<00:00, 3673708.24B/s]


In [42]:
sample_sentence = "Where can I find a pizzeria?"
tokens = bpemb_en.encode(sample_sentence)
print(tokens)

['▁where', '▁can', '▁i', '▁find', '▁a', '▁p', 'iz', 'zer', 'ia', '?']


In [43]:
token_seq = np.array(bpemb_en.encode_ids("Where can I find a pizzeria?"))
print(token_seq)

[ 571  280  386 1934    4   24  248 4339  177 9967]


In [46]:
bpemb_vocab_size, bpemb_embed_size = bpemb_en.vectors.shape

In [50]:
token_embed = tf.keras.layers.Embedding(bpemb_vocab_size, embed_dim)
token_embeddings = token_embed(token_seq)

# The untrained embeddings for our sample sentence.
print("Embeddings for: ", sample_sentence)
print(token_embeddings)

Embeddings for:  Where can I find a pizzeria?
tf.Tensor(
[[-0.02258208  0.01114035 -0.03177005  0.03801126 -0.03929411 -0.03286447
  -0.00218701  0.00356279 -0.03534804 -0.02564455 -0.04835266 -0.01086277]
 [ 0.01455561 -0.04867085 -0.00847696 -0.02035747 -0.040521    0.03266202
   0.0439996   0.03630202 -0.04357444 -0.0140309  -0.02653326  0.04436481]
 [-0.03320471  0.01401706  0.01552368 -0.02872876 -0.01238068  0.00583403
   0.0386689  -0.04964479 -0.03199156 -0.02197343  0.0231226   0.04751891]
 [-0.03421488 -0.04697162  0.01468997  0.03280308 -0.0119614  -0.00092625
   0.02728098 -0.02973909 -0.0060048  -0.03663726 -0.00198963 -0.04671491]
 [ 0.02037365  0.02644915  0.01711626 -0.00621597  0.00655352 -0.03800426
  -0.00190554 -0.02403221  0.00326009 -0.00012296  0.01326806  0.00483131]
 [ 0.0389953   0.04068813  0.0178881  -0.01872829  0.02109568 -0.03778852
   0.00726242  0.02649016  0.0428809  -0.04017204  0.04011536 -0.00675756]
 [ 0.03222758  0.04906735 -0.03853768 -0.01601539

In [49]:
bpemb_vocab_size

10000

In [51]:
embed_dim

12

In [55]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_blocks, d_model, num_heads, hidden_dim, src_vocab_size,
               max_seq_len, dropout_rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.max_seq_len = max_seq_len

    self.token_embed = tf.keras.layers.Embedding(src_vocab_size, self.d_model)
    self.pos_embed = tf.keras.layers.Embedding(max_seq_len, self.d_model)

    # The original Attention Is All You Need paper applied dropout to the
    # input before feeding it to the first encoder block.
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

    # Create encoder blocks.
    self.blocks = [EncoderBlock(self.d_model, num_heads, hidden_dim, dropout_rate) 
    for _ in range(num_blocks)]
  
  def calld(self, input, training, mask):
    token_embeds = self.token_embed(input)

    # Generate position indices for a batch of input sequences.
    num_pos = input.shape[0] * self.max_seq_len
    pos_idx = np.resize(np.arange(self.max_seq_len), num_pos)
    pos_idx = np.reshape(pos_idx, input.shape)
    pos_embeds = self.pos_embed(pos_idx)

    x = self.dropout(token_embeds + pos_embeds, training=training)

    # Run input through successive encoder blocks.
    for block in self.blocks:
      x, weights = block(x, training, mask)

    return x, weights

In [56]:
class DecoderBlock(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, hidden_dim, dropout_rate=0.1):
    super(DecoderBlock, self).__init__()

    self.mhsa1 = MultiHeadSelfAttention(d_model, num_heads)
    self.mhsa2 = MultiHeadSelfAttention(d_model, num_heads)

    self.ffn = feed_forward_network(d_model, hidden_dim)

    self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
    self.dropout2 = tf.keras.layers.Dropout(dropout_rate)
    self.dropout3 = tf.keras.layers.Dropout(dropout_rate)

    self.layernorm1 = tf.keras.layers.LayerNormalization()
    self.layernorm2 = tf.keras.layers.LayerNormalization()
    self.layernorm3 = tf.keras.layers.LayerNormalization()
  
  # Note the decoder block takes two masks. One for the first MHSA, another
  # for the second MHSA.
  def call(self, encoder_output, target, training, decoder_mask, memory_mask):
    mhsa_output1, attn_weights = self.mhsa1(target, target, target, decoder_mask)
    mhsa_output1 = self.dropout1(mhsa_output1, training=training)
    mhsa_output1 = self.layernorm1(mhsa_output1 + target)

    mhsa_output2, attn_weights = self.mhsa2(mhsa_output1, encoder_output, 
                                            encoder_output, 
                                            memory_mask)
    mhsa_output2 = self.dropout2(mhsa_output2, training=training)
    mhsa_output2 = self.layernorm2(mhsa_output2 + mhsa_output1)

    ffn_output = self.ffn(mhsa_output2)
    ffn_output = self.dropout3(ffn_output, training=training)
    output = self.layernorm3(ffn_output + mhsa_output2)

    return output, attn_weights


In [57]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
               max_seq_len, dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.max_seq_len = max_seq_len

    self.token_embed = tf.keras.layers.Embedding(target_vocab_size, self.d_model)
    self.pos_embed = tf.keras.layers.Embedding(max_seq_len, self.d_model)

    self.dropout = tf.keras.layers.Dropout(dropout_rate)

    self.blocks = [DecoderBlock(self.d_model, num_heads, hidden_dim, dropout_rate) for _ in range(num_blocks)]

  def call(self, encoder_output, target, training, decoder_mask, memory_mask):
    token_embeds = self.token_embed(target)

    # Generate position indices.
    num_pos = target.shape[0] * self.max_seq_len
    pos_idx = np.resize(np.arange(self.max_seq_len), num_pos)
    pos_idx = np.reshape(pos_idx, target.shape)

    pos_embeds = self.pos_embed(pos_idx)

    x = self.dropout(token_embeds + pos_embeds, training=training)

    for block in self.blocks:
      x, weights = block(encoder_output, x, training, decoder_mask, memory_mask)

    return x, weights

In [58]:
class Transformer(tf.keras.Model):
  def __init__(self, num_blocks, d_model, num_heads, hidden_dim, source_vocab_size,
               target_vocab_size, max_input_len, max_target_len, dropout_rate=0.1):
    super(Transformer, self).__init__()

    self.encoder = Encoder(num_blocks, d_model, num_heads, hidden_dim, source_vocab_size, 
                           max_input_len, dropout_rate)
    
    self.decoder = Decoder(num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
                           max_target_len, dropout_rate)
    
    # The final dense layer to generate logits from the decoder output.
    self.output_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, input_seqs, target_input_seqs, training, encoder_mask,
           decoder_mask, memory_mask):
    encoder_output, encoder_attn_weights = self.encoder(input_seqs, 
                                                        training, encoder_mask)

    decoder_output, decoder_attn_weights = self.decoder(encoder_output, 
                                                        target_input_seqs, training,
                                                        decoder_mask, memory_mask)

    return self.output_layer(decoder_output), encoder_attn_weights, decoder_attn_weights


In [62]:
input_batch = [
    "Where can I find a pizzeria?",
    "Mass hysteria over listeria.",
    "I ain't no circle back girl."
]

bpemb_en.encode(input_batch)

[['▁where', '▁can', '▁i', '▁find', '▁a', '▁p', 'iz', 'zer', 'ia', '?'],
 ['▁mass', '▁hy', 'ster', 'ia', '▁over', '▁l', 'ister', 'ia', '.'],
 ['▁i', '▁a', 'in', "'", 't', '▁no', '▁circle', '▁back', '▁girl', '.']]

In [63]:
input_seqs = bpemb_en.encode_ids(input_batch)
print("Vectorized inputs:")
input_seqs

Vectorized inputs:


[[571, 280, 386, 1934, 4, 24, 248, 4339, 177, 9967],
 [1535, 1354, 1238, 177, 380, 43, 871, 177, 9935],
 [386, 4, 6, 9937, 9915, 467, 5410, 810, 3692, 9935]]

In [64]:

padded_input_seqs = tf.keras.preprocessing.sequence.pad_sequences(input_seqs, padding="post")
print("Input to the encoder:")
print(padded_input_seqs.shape)
print(padded_input_seqs)

Input to the encoder:
(3, 10)
[[ 571  280  386 1934    4   24  248 4339  177 9967]
 [1535 1354 1238  177  380   43  871  177 9935    0]
 [ 386    4    6 9937 9915  467 5410  810 3692 9935]]


In [67]:
# Made up values.
target_input_seqs = [
    [1, 652, 723, 123, 62],
    [1, 25,  98, 129, 248, 215, 359, 249],
    [1, 2369, 1259, 125, 486],
]
padded_target_input_seqs = tf.keras.preprocessing.sequence.pad_sequences(target_input_seqs, padding="post")
print("Padded target inputs to the decoder:")
print(padded_target_input_seqs.shape)
print(padded_target_input_seqs)

Padded target inputs to the decoder:
(3, 8)
[[   1  652  723  123   62    0    0    0]
 [   1   25   98  129  248  215  359  249]
 [   1 2369 1259  125  486    0    0    0]]


In [68]:
transformer = Transformer(
    num_blocks = 6,
    d_model = 12,
    num_heads = 3,
    hidden_dim = 48,
    source_vocab_size = bpemb_vocab_size,
    target_vocab_size = 7000, # made-up target vocab size.
    max_input_len = padded_input_seqs.shape[1],
    max_target_len = padded_target_input_seqs.shape[1])

transformer_output, _, _ = transformer(padded_input_seqs, 
                                       padded_target_input_seqs, True, 
                                       enc_mask, dec_mask, memory_mask=enc_mask)
print(f"Transformer output {transformer_output.shape}:")
print(transformer_output) # If training, we would use this output to calculate losses.

NameError: name 'enc_mask' is not defined